![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 1. Clinical Named Entity Recognition (NER) v2.7.0

Please make sure that your cluster is setup properly according to https://nlp.johnsnowlabs.com/docs/en/licensed_install#install-spark-nlp-for-healthcare-on-databricks

## Blogposts and videos:

https://towardsdatascience.com/named-entity-recognition-ner-with-bert-in-spark-nlp-874df20d1d77

https://www.youtube.com/watch?v=YM-e4eOiQ34

https://medium.com/spark-nlp/named-entity-recognition-for-healthcare-with-sparknlp-nerdl-and-nercrf-a7751b6ad571

https://medium.com/atlas-research/ner-for-clinical-text-7c73caddd180

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

2.6.3

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.246.233:7077 
 AppName 
 Databricks Shell

# Clinical NER Pipeline (with pretrained models)

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

#sentenceDetector = SentenceDetector()\
  #.setInputCols(["document"])\
  #.setOutputCol("sentence")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
	.setInputCols(["document"]) \
	.setOutputCol("sentence") 
 
# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = NerDLModel.pretrained("ner_clinical_large", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 321.1 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
! wget -q https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv

  

In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"""Monomeric sarcosine oxidase (MSOX) is a flav...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [0]:
pubMedDF.printSchema()

root
-- text: string (nullable = true)

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 48, 106,...|
BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 67, 79, ...|
OBJECTIVE: To inv...|[[document, 0, 15...|[[document, 0, 14...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 130, 144...|
Combined EEG/fMRI...|[[document, 0, 16...|[[document, 0, 29...|[[token, 0, 7, Co...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 26, C...|
Kohlschutter synd...|[[document, 0, 25...|[[document, 0, 20...|[[token, 0, 11, K...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 20, K...|
Statistical analy...|[[document, 0, 10...|[[document, 0, 22...|[[token, 0, 10, S...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 34, S...|
The synthetic DOX...|[[document, 0, 57...|[[document, 0, 10...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 30, T...|
Our objective was...|[[document, 0, 24...|[[document, 0, 19...|[[token, 0, 2, Ou...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 56, 80, ...|
We conducted a ph...|[[document, 0, 14...|[[document, 0, 24...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 28, ...|
"""Monomeric sarc...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 3, 36, M...|
We presented the ...|[[document, 0, 12...|[[document, 0, 34...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 46, ...|
The literature de...|[[document, 0, 16...|[[document, 0, 13...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 154, 166...|
A novel approach ...|[[document, 0, 64...|[[document, 0, 97...|[[token, 0, 0, A,...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 99, 124,...|
An HPLC-ESI-MS-MS...|[[document, 0, 90...|[[document, 0, 24...|[[token, 0, 1, An...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 23, A...|
The localizing an...|[[document, 0, 72...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 50, 70, ...|
OBJECTIVE: To eva...|[[document, 0, 13...|[[document, 0, 14...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 97, 143,...|
For the construct...|[[document, 0, 32...|[[document, 0, 20...|[[token, 0, 2, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 24, 50, ...|
We report the res...|[[document, 0, 13...|[[document, 0, 31...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 25, 32, ...|
Intraparenchymal ...|[[document, 0, 10...|[[document, 0, 53...|[[token, 0, 15, I...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 33, I...|
It is known that ...|[[document, 0, 34...|[[document, 0, 14...|[[token, 0, 1, It...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 31, 52, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[The, human, KCNJ9, (, Kir, 3.3, ,, GIRK3, ), is, a, member, of, the, G-protein-activated, inward...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TRE...|
[BACKGROUND, :, At, present, ,, it, is, one, of, the, most, important, issues, for, the, treatmen...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, B-PROBLEM, I-PROBLEM, O, ...|
[OBJECTIVE, :, To, investigate, the, relationship, between, preoperative, atrialfibrillation, and...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, O, O, O, O, O, B...|
[Combined, EEG/fMRI, recording, has, been, used, to, localize, the, generators, of, EEGevents, an...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-TEST, I-TEST, O, O, O, O,...|
[Kohlschutter, syndrome, is, a, rare, neurodegenerative, disorder, presenting, withintractable, s...|[B-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, O,...|
[Statistical, analysis, of, neuroimages, is, commonly, approached, with, intergroupcomparisons, m...|[B-TEST, I-TEST, I-TEST, I-TEST, O, O, O, O, B-TREATMENT, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, synthetic, DOX-LNA, conjugate, was, characterized, by, proton, nuclear, magneticresonance, ...|[B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBL...|
[Our, objective, was, to, compare, three, different, methods, of, blood, pressuremeasurement, thr...|[O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, O, O, B-TEST, I-TEST, I-TEST, O, O, O...|
[We, conducted, a, phase, II, study, to, assess, the, efficacy, and, tolerability, ofirinotecan, ...|[O, O, B-TEST, I-TEST, I-TEST, I-TEST, O, O, B-TREATMENT, I-TREATMENT, O, B-TREATMENT, I-TREATMEN...|
[""", Monomeric, sarcosine, oxidase, (, MSOX, ), is, a, flavoenzyme, that, catalyzes, the, oxidat...|[O, B-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, I-TREATMENT, O, B-TREATMENT,...|
[We, presented, the, tachinid, fly, Exorista, japonica, with, moving, host, models, :, afreeze-dr...|[O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, O, O, O, B-PROBLEM, ...|
[The, literature, dealing, with, the, water, conducting, properties, of, sapwood, xylem, intrees,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, O, O, O, O, O...|
[A, novel, approach, to, synthesize, chitosan-O-isopropyl-5'-O-d4T, monophosphateconjugate, was, ...|[O, O, O, O, O, O, O, O, O, O, B-TREATMENT, I-TREATMENT, O, O, B-PROBLEM, I-PROBLEM, O, O, O, O, ...|
[An, HPLC-ESI-MS-MS, method, has, been, developed, for, the, quantitative, determination, offour,...|[B-TEST, I-TEST, I-TEST, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, O, B-TREATMENT, I-TREATMENT, ...|
[The, localizing, and, lateralizing, values, of, eye, and, head, ictal, deviations, duringfrontal...|[O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, ...|
[OBJECTIVE, :, To, evaluate, the, effectiveness, and, acceptability, of, expectantmanagement, of,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, I-PROBLEM, O,...|
[For, the, construction, of, new, combinatorial, libraries, ,, a, lead, compound, was, created, b...|[O, O, O, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, B-PROBLEM, I-PROBLEM, I-PROBLEM, O, O, O, O, O, ...|
[We, report, the, results, of, a, screen, for, genetic, association, with, urinary, arsenicmetabo...|[O, O, O, O, O, B-TEST,

In [0]:
result_df = result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))

result_df.show(50, truncate=100)


+-------------------+-----------+
 token| ner_label|
+-------------------+-----------+
 The| O|
 human| O|
 KCNJ9| O|
 (| O|
 Kir| O|
 3.3| O|
 ,| O|
 GIRK3| O|
 )| O|
 is| O|
 a| O|
 member| O|
 of| O|
 the|B-TREATMENT|
G-protein-activated|I-TREATMENT|
 inwardly|I-TREATMENT|
 rectifying|I-TREATMENT|
 potassium|I-TREATMENT|
 (|I-TREATMENT|
 GIRK|I-TREATMENT|
 )| O|
 channel| O|
 family| O|
 .| O|
 Here| O|
 we| O|
 describe| O|
 the|B-TREATMENT|
genomicorganization|I-TREATMENT|
 of| O|
 the| O|
 KCNJ9| O|
 locus| O|
 on| O|
 chromosome| O|
 1q21-23| O|
 as| O|
 a| B-PROBLEM|
 candidate| I-PROBLEM|
 gene| I-PROBLEM|
 forType| I-PROBLEM|
 II| I-PROBLEM|
 diabetes| I-PROBLEM|
 mellitus| I-PROBLEM|
 in| O|
 the| O|
 Pima| O|
 Indian| O|
 population| O|
 .| O|
+-------------------+-----------+
only showing top 50 rows

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+-----------+-----+
ner_label |count|
+-----------+-----+
O |13868|
I-PROBLEM |1591 |
B-PROBLEM |914 |
I-TREATMENT|739 |
I-TEST |632 |
B-TREATMENT|622 |
B-TEST |534 |
+-----------+-----+

In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------------------------------------------+---------+
chunk |ner_label|
+-----------------------------------------------------------+---------+
the G-protein-activated inwardly rectifying potassium (GIRK|TREATMENT|
the genomicorganization |TREATMENT|
a candidate gene forType II diabetes mellitus |PROBLEM |
byapproximately |TREATMENT|
single nucleotide polymorphisms |TREATMENT|
aVal366Ala substitution |PROBLEM |
an 8 base-pair |PROBLEM |
insertion/deletion |PROBLEM |
Ourexpression studies |TEST |
the transcript in various humantissues |PROBLEM |
fat andskeletal muscle |PROBLEM |
furtherstudies |PROBLEM |
the KCNJ9 protein |TREATMENT|
evaluation |TEST |
Type II diabetes |PROBLEM |
the treatment |TREATMENT|
breast cancer |PROBLEM |
the standard therapy |TREATMENT|
anthracyclines |TREATMENT|
taxanes |TREATMENT|
+-----------------------------------------------------------+---------+
only showing top 20 rows

In [0]:
# fullAnnotate in LightPipeline

text = '''
A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus ( T2DM ), one prior episode of HTG-induced pancreatitis three years prior to presentation , associated with an acute hepatitis , and obesity with a body mass index ( BMI ) of 33.5 kg/m2 , presented with a one-week history of polyuria , polydipsia , poor appetite , and vomiting . Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation . Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness , guarding , or rigidity . Pertinent laboratory findings on admission were : serum glucose 111 mg/dl , bicarbonate 18 mmol/l , anion gap 20 , creatinine 0.4 mg/dL , triglycerides 508 mg/dL , total cholesterol 122 mg/dL , glycated hemoglobin ( HbA1c ) 10% , and venous pH 7.27 . Serum lipase was normal at 43 U/L . Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia . The patient was initially admitted for starvation ketosis , as she reported poor oral intake for three days prior to admission . However , serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL , the anion gap was still elevated at 21 , serum bicarbonate was 16 mmol/L , triglyceride level peaked at 2050 mg/dL , and lipase was 52 U/L . The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again . The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL , within 24 hours . Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use . The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely . She had close follow-up with endocrinology post discharge .
'''

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    sentence.append(n.metadata['sentence'])
    
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 
                   'sentence_id':sentence, 'entities':entities})

df.head(20)

,chunks,begin,end,sentence_id,entities
0,gestational diabetes mellitus,40,68,0,PROBLEM
1,subsequent type two diabetes mellitus,118,154,0,PROBLEM
2,T2DM,158,161,0,PROBLEM
3,HTG-induced pancreatitis,187,210,0,PROBLEM
4,an acute hepatitis,264,281,0,PROBLEM
5,obesity,289,295,0,PROBLEM
6,a body mass index,302,318,0,PROBLEM
7,BMI,322,324,0,TEST
8,polyuria,381,388,0,PROBLEM
9,polydipsia,392,401,0,PROBLEM


## Posology NER

In [0]:
# NER model trained on i2b2 (sampled from MIMIC) dataset
posology_ner = NerDLModel.pretrained("ner_posology", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")


nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    posology_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

posology_model = nlpPipeline.fit(empty_data)


ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
posology_result = posology_model.transform(pubMedDF.limit(100))

In [0]:
posology_result.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...| []|
BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 167, 180...|
OBJECTIVE: To inv...|[[document, 0, 15...|[[document, 0, 14...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...| []|
Combined EEG/fMRI...|[[document, 0, 16...|[[document, 0, 29...|[[token, 0, 7, Co...|[[word_embeddings...|[[named_entity, 0...| []|
Kohlschutter synd...|[[document, 0, 25...|[[document, 0, 20...|[[token, 0, 11, K...|[[word_embeddings...|[[named_entity, 0...| []|
Statistical analy...|[[document, 0, 10...|[[document, 0, 22...|[[token, 0, 10, S...|[[word_embeddings...|[[named_entity, 0...| []|
The synthetic DOX...|[[document, 0, 57...|[[document, 0, 10...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 432, 446...|
Our objective was...|[[document, 0, 24...|[[document, 0, 19...|[[token, 0, 2, Ou...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 174, 185...|
We conducted a ph...|[[document, 0, 14...|[[document, 0, 24...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 70, 81, ...|
"""Monomeric sarc...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 138, 144...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [0]:
posology_result.printSchema()

root
-- text: string (nullable = true)
-- document: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- sentence: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- token: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- embeddings: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)
-- ner_chunk: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- annotatorType: string (nullable = true)
 | |-- begin: integer (nullable = false)
 | |-- end: integer (nullable = false)
 | |-- result: string (nullable = true)
 | |-- metadata: map (nullable = true)
 | | |-- key: string
 | | |-- value: string (valueContainsNull = true)
 | |-- embeddings: array (nullable = true)
 | | |-- element: float (containsNull = false)

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# This will return a new DF with all the columns + id
posology_result = posology_result.withColumn("id", monotonically_increasing_id())

posology_result.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
 text| document| sentence| token| embeddings| ner| ner_chunk| id|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...| []| 0|
BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 167, 180...| 1|
OBJECTIVE: To inv...|[[document, 0, 15...|[[document, 0, 14...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...| []| 2|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 3 rows

In [0]:
posology_result.select('token.result','ner.result').show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
 result| result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
[The, human, KCNJ9, (, Kir, 3.3, ,, GIRK3, ), is, a, member, of, the, G-protein-activated, inward...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[BACKGROUND, :, At, present, ,, it, is, one, of, the, most, important, issues, for, the, treatmen...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O...|
[OBJECTIVE, :, To, investigate, the, relationship, between, preoperative, atrialfibrillation, and...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Combined, EEG/fMRI, recording, has, been, used, to, localize, the, generators, of, EEGevents, an...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Kohlschutter, syndrome, is, a, rare, neurodegenerative, disorder, presenting, withintractable, s...| [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]|
[Statistical, analysis, of, neuroimages, is, commonly, approached, with, intergroupcomparisons, m...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, synthetic, DOX-LNA, conjugate, was, characterized, by, proton, nuclear, magneticresonance, ...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[Our, objective, was, to, compare, three, different, methods, of, blood, pressuremeasurement, thr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O, B-DRUG, O, O,...|
[We, conducted, a, phase, II, study, to, assess, the, efficacy, and, tolerability, ofirinotecan, ...|[O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, O, B-DRUG, O, O, B-DRUG, O, O, O, O, O, O, O, O, O, ...|
[""", Monomeric, sarcosine, oxidase, (, MSOX, ), is, a, flavoenzyme, that, catalyzes, the, oxidat...| [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG]|
[We, presented, the, tachinid, fly, Exorista, japonica, with, moving, host, models, :, afreeze-dr...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[The, literature, dealing, with, the, water, conducting, properties, of, sapwood, xylem, intrees,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[A, novel, approach, to, synthesize, chitosan-O-isopropyl-5'-O-d4T, monophosphateconjugate, was, ...|[O, O, O, O, O, O, O, O, O, O, B-DRUG, I-DRUG, I-DRUG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[An, HPLC-ESI-MS-MS, method, has, been, developed, for, the, quantitative, determination, offour,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, B-DRUG, I-DRUG, I-DRUG, I-DRUG, O, O, O, O, O, O, O, O, O...|
[The, localizing, and, lateralizing, values, of, eye, and, head, ictal, deviations, duringfrontal...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[OBJECTIVE, :, To, evaluate, the, effectiveness, and, acceptability, of, expectantmanagement, of,...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[For, the, construction, of, new, combinatorial, libraries, ,, a, lead, compound, was, created, b...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...|
[We, report, the, results, of, a, screen, for, genetic, association, with, urinary, arsenicmetabo...|[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O,

In [0]:
posology_result.select('token.result','ner.result').take(2)

Out[23]: [Row(result=['The', 'human', 'KCNJ9', '(', 'Kir', '3.3', ',', 'GIRK3', ')', 'is', 'a', 'member', 'of', 'the', 'G-protein-activated', 'inwardly', 'rectifying', 'potassium', '(', 'GIRK', ')', 'channel', 'family', '.', 'Here', 'we', 'describe', 'the', 'genomicorganization', 'of', 'the', 'KCNJ9', 'locus', 'on', 'chromosome', '1q21-23', 'as', 'a', 'candidate', 'gene', 'forType', 'II', 'diabetes', 'mellitus', 'in', 'the', 'Pima', 'Indian', 'population', '.', 'The', 'gene', 'spansapproximately', '7.6', 'kb', 'and', 'contains', 'one', 'noncoding', 'and', 'two', 'coding', 'exons', 'separated', 'byapproximately', '2.2', 'and', 'approximately', '2.6', 'kb', 'introns', ',', 'respectively', '.', 'We', 'identified14', 'single', 'nucleotide', 'polymorphisms', '(', 'SNPs', '),', 'including', 'one', 'that', 'predicts', 'aVal366Ala', 'substitution', ',', 'and', 'an', '8', 'base-pair', '(', 'bp', ')', 'insertion/deletion', '.', 'Ourexpression', 'studies', 'revealed', 'the', 'presence', 'of', 'the', 'transcript', 'in', 'various', 'humantissues', 'including', 'pancreas', ',', 'and', 'two', 'major', 'insulin-responsive', 'tissues', ':', 'fat', 'andskeletal', 'muscle', '.', 'The', 'characterization', 'of', 'the', 'KCNJ9', 'gene', 'should', 'facilitate', 'furtherstudies', 'on', 'the', 'function', 'of', 'the', 'KCNJ9', 'protein', 'and', 'allow', 'evaluation', 'of', 'thepotential', 'role', 'of', 'the', 'locus', 'in', 'Type', 'II', 'diabetes', '.'], result=['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
 Row(result=['BACKGROUND', ':', 'At', 'present', ',', 'it', 'is', 'one', 'of', 'the', 'most', 'important', 'issues', 'for', 'the', 'treatment', 'of', 'breast', 'cancer', 'to', 'develop', 'the', 'standard', 'therapy', 'for', 'patients', 'previously', 'treated', 'with', 'anthracyclines', 'and', 'taxanes', '.', 'With', 'the', 'objective', 'of', 'determining', 'the', 'usefulnessof', 'vinorelbine', 'monotherapy', 'in', 'patients', 'with', 'advanced', 'or', 'recurrent', 'breast', 'cancerafter', 'standard', 'therapy', ',', 'we', 'evaluated', 'the', 'efficacy', 'and', 'safety', 'of', 'vinorelbine', 'inpatients', 'previously', 'treated', 'with', 'anthracyclines', 'and', 'taxanes', '.', 'METHODS', ':', 'Vinorelbinewas', 'administered', 'at', 'a', 'dose', 'level', 'of', '25', 'mg/m(2', ')', 'intravenously', 'on', 'days', '1', 'and', '8', 'of', 'a3', 'week', 'cycle', '.', 'Patients', 'were', 'given', 'three', 'or', 'more', 'cycles', 'in', 'the', 'absence', 'of', 'tumorprogression', '.', 'A', 'maximum', 'of', 'nine', 'cycles', 'were', 'administered', '.', 'RESULTS', ':', 'The', 'responserate', 'in', '50', 'evaluable', 'patients', 'was', '20.0%', '(', '10', 'out', 'of', '50', ';', '95%', 'confidence', 'interval,10.0-33.7%', ').', 'Responders', 'plus', 'those', 'who', 'had', 'minor', 'response', '(', 'MR', ')', 'or', 'no', 'change', '(', 'NC', ')', 'accounted', 'for', '58.0%', '[10', 'partial', 'responses', '(', 'PRs', ')', '+', 'one', 'MR', '+', '18', 'NCs', 'out', 'of', '50]', '.', 'TheKaplan-Meier', 'estimate', '(', '50%', 'point', ')', 'of', 'time', 'to', 'progression', '(', 'TTP', ')', 'was', '115.0', 'days', '.', 'The', 'response', 'rate', 'in', 'the', 'visceral', 'organs', 'was', '17.3%', '(', 'nine', 'PRs', 'out', 'of', '52', ').', 'Themajor', 'toxici

In [0]:
from pyspark.sql import functions as F

posology_result_df = posology_result.select(F.explode(F.arrays_zip('token.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label"))\
        .filter("ner_label!='O'")

posology_result_df.show(20, truncate=100)


+----------------+-----------+
 token| ner_label|
+----------------+-----------+
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 vinorelbine| B-DRUG|
 vinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 Vinorelbinewas| B-DRUG|
 25| B-STRENGTH|
 mg/m(2| I-STRENGTH|
 intravenously| B-ROUTE|
 on|B-FREQUENCY|
 days|I-FREQUENCY|
tumorprogression| B-DRUG|
 thatvinorelbine| B-DRUG|
 anthracyclines| B-DRUG|
 taxanes| B-DRUG|
 tetrazolium| B-DRUG|
 dye| I-DRUG|
 DOX| B-DRUG|
 trandolapril| B-DRUG|
+----------------+-----------+
only showing top 20 rows

In [0]:
posology_result.select('id',F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata')).alias("cols")) \
.select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
        F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"))\
        .filter("ner_label!='O'")\
        .show(truncate=False)

+---+-----------+----------------+-----+----+---------+
id |sentence_id|chunk |begin|end |ner_label|
+---+-----------+----------------+-----+----+---------+
1 |0 |anthracyclines |167 |180 |DRUG |
1 |0 |taxanes |186 |192 |DRUG |
1 |1 |vinorelbine |246 |256 |DRUG |
1 |1 |vinorelbine |386 |396 |DRUG |
1 |1 |anthracyclines |433 |446 |DRUG |
1 |1 |taxanes |452 |458 |DRUG |
1 |2 |Vinorelbinewas |470 |483 |DRUG |
1 |2 |25 mg/m(2 |517 |525 |STRENGTH |
1 |2 |intravenously |528 |540 |ROUTE |
1 |2 |on days |542 |548 |FREQUENCY|
1 |3 |tumorprogression|635 |650 |DRUG |
1 |10 |thatvinorelbine |1263 |1277|DRUG |
1 |10 |anthracyclines |1380 |1393|DRUG |
1 |10 |taxanes |1399 |1405|DRUG |
6 |2 |tetrazolium dye |432 |446 |DRUG |
6 |3 |DOX |576 |578 |DRUG |
7 |0 |trandolapril |174 |185 |DRUG |
7 |0 |losartan |191 |198 |DRUG |
7 |2 |3-week |313 |318 |DURATION |
7 |2 |placebo |320 |326 |DRUG |
+---+-----------+----------------+-----+----+---------+
only showing top 20 rows

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].result

Out[26]: 'anthracyclines'

In [0]:
posology_result.select('ner_chunk').take(2)[1][0][0].metadata

Out[27]: {'sentence': '0', 'chunk': '0', 'entity': 'DRUG'}

### with LightPipelines

In [0]:
light_model = LightPipeline(posology_model)

text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

list(zip(light_result['token'], light_result['ner']))

Out[28]: [('The', 'O'),
 ('patient', 'O'),
 ('was', 'O'),
 ('prescribed', 'O'),
 ('1', 'B-DOSAGE'),
 ('capsule', 'B-FORM'),
 ('of', 'O'),
 ('Advil', 'B-DRUG'),
 ('for', 'B-DURATION'),
 ('5', 'I-DURATION'),
 ('days', 'I-DURATION'),
 ('.', 'O'),
 ('He', 'O'),
 ('was', 'O'),
 ('seen', 'O'),
 ('by', 'O'),
 ('the', 'O'),
 ('endocrinology', 'O'),
 ('service', 'O'),
 ('and', 'O'),
 ('she', 'O'),
 ('was', 'O'),
 ('discharged', 'O'),
 ('on', 'O'),
 ('40', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('glargine', 'I-DRUG'),
 ('at', 'B-FREQUENCY'),
 ('night', 'I-FREQUENCY'),
 (',', 'O'),
 ('12', 'B-DOSAGE'),
 ('units', 'I-DOSAGE'),
 ('of', 'O'),
 ('insulin', 'B-DRUG'),
 ('lispro', 'I-DRUG'),
 ('with', 'B-FREQUENCY'),
 ('meals', 'I-FREQUENCY'),
 (',', 'O'),
 ('and', 'O'),
 ('metformin', 'B-DRUG'),
 ('1000', 'B-STRENGTH'),
 ('mg', 'I-STRENGTH'),
 ('two', 'B-FREQUENCY'),
 ('times', 'I-FREQUENCY'),
 ('a', 'I-FREQUENCY'),
 ('day', 'I-FREQUENCY'),
 ('.', 'O'),
 ('It', 'O'),
 ('was', 'O'),
 ('determined', 'O'),
 ('that', 'O'),
 ('all', 'O'),
 ('SGLT2', 'B-DRUG'),
 ('inhibitors', 'I-DRUG'),
 ('should', 'O'),
 ('be', 'O'),
 ('discontinued', 'O'),
 ('indefinitely', 'O'),
 ('fro', 'O'),
 ('3', 'O'),
 ('months', 'O'),
 ('.', 'O')]

In [0]:
light_result = light_model.fullAnnotate(text)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:
        
    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    
import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                   'begin': begin, 'end': end})

df

,chunks,entities,begin,end
0,1,DOSAGE,27,27
1,capsule,FORM,29,35
2,Advil,DRUG,40,44
3,for 5 days,DURATION,46,55
4,40 units,DOSAGE,126,133
5,insulin glargine,DRUG,138,153
6,at night,FREQUENCY,155,162
7,12 units,DOSAGE,166,173
8,insulin lispro,DRUG,178,191
9,with meals,FREQUENCY,193,202


#### NER Highlighter

In [0]:
import sys
import ner_highlighter

# Add the path to system, local or mounted S3 bucket, e.g. /dbfs/mnt/<path_to_bucket>
sys.path.append('/databricks/driver/')
sys.path.append('/databricks/driver/ner_highlighter.py')



In [0]:
text ='The patient was prescribed 1 capsule of Advil for 5 days . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_result = light_model.annotate(text)

displayHTML(ner_highlighter.chunk_highlighter (light_result))

The patient was prescribed 
 
 1
 DOSAGE 
 
 
 
 capsule
 FORM 
 
 of 
 
 Advil
 DRUG 
 
 
 
 for 5 days
 DURATION 
 
 . He was seen by the endocrinology service and she was discharged on 
 
 40 units
 DOSAGE 
 
 of 
 
 insulin glargine
 DRUG 
 
 
 
 at night
 FREQUENCY 
 
 , 
 
 12 units
 DOSAGE 
 
 of 
 
 insulin lispro
 DRUG 
 
 
 
 with meals
 FREQUENCY 
 
 , and 
 
 metformin
 DRUG 
 
 
 
 1000 mg
 STRENGTH 
 
 
 
 two times a day
 FREQUENCY 
 
 . It was determined that all 
 
 SGLT2 inhibitors
 DRUG 
 
 should be discontinued indefinitely fro 3 months .

## Writing a generic NER function

In [0]:
def get_base_pipeline (embeddings = 'embeddings_clinical'):

  documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

  sentenceDetector = SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

  tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
  word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  return model



def get_clinical_entities (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
  loaded_ner_model = NerDLModel.pretrained(model_name, "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

  ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")

  base_model = get_base_pipeline (embeddings)

  nlpPipeline = Pipeline(stages=[
      base_model,
      loaded_ner_model,
      ner_converter])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  model = nlpPipeline.fit(empty_data)

  result = model.transform(spark_df.limit(nrows))

  result = result.withColumn("id", monotonically_increasing_id())

  result_df = result.select('id',F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata')).alias("cols")) \
  .select('id', F.expr("cols['3']['sentence']").alias("sentence_id"),
          F.expr("cols['0']").alias("chunk"),
          F.expr("cols['1']").alias("begin"),
          F.expr("cols['2']").alias("end"),
          F.expr("cols['3']['entity']").alias("ner_label"))\
          .filter("ner_label!='O'")

  return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+--------------------+-----+---+---------+
 id|sentence_id| chunk|begin|end|ner_label|
+---+-----------+--------------------+-----+---+---------+
 0| 0| Kir| 17| 19| TEST|
 0| 0| GIRK3| 26| 30| TEST|
 0| 1| chromosome 1q21-23| 188|205|TREATMENT|
 0| 1|a candidate gene ...| 210|254| PROBLEM|
 0| 2| coding exons| 357|368| TEST|
 0| 2| byapproximately| 380|394| TEST|
 0| 2| introns| 425|431| TEST|
 0| 3|single nucleotide...| 464|494|TREATMENT|
 0| 3| SNPs| 497|500| PROBLEM|
 0| 3|aVal366Ala substi...| 532|554|TREATMENT|
 0| 3| an 8 base-pair| 561|574| TEST|
 0| 3| bp| 577|578| TEST|
 0| 3| insertion/deletion| 581|598| PROBLEM|
 0| 4|Ourexpression stu...| 601|621| TEST|
 0| 5|The characterizat...| 773|810| TEST|
 0| 5| furtherstudies| 830|843|TREATMENT|
 0| 5| KCNJ9 protein| 868|880| TEST|
 0| 5| evaluation| 892|901| TEST|
 0| 5| Type II diabetes| 940|955| PROBLEM|
 1| 0| the treatment| 67| 79|TREATMENT|
+---+-----------+--------------------+-----+---+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.7 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+----------------+-----+----+---------+
 id|sentence_id| chunk|begin| end|ner_label|
+---+-----------+----------------+-----+----+---------+
 1| 0| anthracyclines| 167| 180| DRUG|
 1| 0| taxanes| 186| 192| DRUG|
 1| 1| vinorelbine| 246| 256| DRUG|
 1| 1| vinorelbine| 386| 396| DRUG|
 1| 1| anthracyclines| 433| 446| DRUG|
 1| 1| taxanes| 452| 458| DRUG|
 1| 2| Vinorelbinewas| 470| 483| DRUG|
 1| 2| 25 mg/m(2| 517| 525| STRENGTH|
 1| 2| intravenously| 528| 540| ROUTE|
 1| 2| on days| 542| 548|FREQUENCY|
 1| 3|tumorprogression| 635| 650| DRUG|
 1| 10| thatvinorelbine| 1263|1277| DRUG|
 1| 10| anthracyclines| 1380|1393| DRUG|
 1| 10| taxanes| 1399|1405| DRUG|
 6| 2| tetrazolium dye| 432| 446| DRUG|
 6| 3| DOX| 576| 578| DRUG|
 7| 0| trandolapril| 174| 185| DRUG|
 7| 0| losartan| 191| 198| DRUG|
 7| 2| 3-week| 313| 318| DURATION|
 7| 2| placebo| 320| 326| DRUG|
+---+-----------+----------------+-----+----+---------+
only showing top 20 rows

In [0]:
import pandas as pd

def get_clinical_entities_light (light_model, text):

  light_result = light_model.fullAnnotate(text)

  chunks = []
  entities = []

  for n in light_result[0]['ner_chunk']:
          
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      
  df = pd.DataFrame({'chunks':chunks, 'entities':entities})

  return df

In [0]:
text ='The patient was prescribed 1 capsle of Parol with meals . He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day . It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months .'

light_model = LightPipeline(posology_model)

get_clinical_entities_light (light_model, text)


,chunks,entities
0,1,DOSAGE
1,capsle,FORM
2,Parol,DRUG
3,with meals,FREQUENCY
4,40 units,DOSAGE
5,insulin glargine,DRUG
6,at night,FREQUENCY
7,12 units,DOSAGE
8,insulin lispro,DRUG
9,with meals,FREQUENCY


## PHI NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_deid_large'

# deidentify_dl
# ner_deid_large

nrows = 1000

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)


ner_deid_large download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
ner_df.show(2)

+---+-----------+-----------+-----+---+---------+
 id|sentence_id| chunk|begin|end|ner_label|
+---+-----------+-----------+-----+---+---------+
 0| 1|Pima Indian| 263|273| LOCATION|
 2| 1| 1995| 271|274| DATE|
+---+-----------+-----------+-----+---+---------+
only showing top 2 rows

In [0]:
pd_ner_df = ner_df.toPandas()

In [0]:
pd_ner_df.sample(20)

,id,sentence_id,chunk,begin,end,ner_label
365,242,5,"1,900",863,867,DATE
793,575,3,Uruguay,454,460,LOCATION
714,504,2,1996,326,329,DATE
453,290,4,RCMEC,561,565,LOCATION
1183,902,2,Spain,603,607,LOCATION
65,44,3,1985,530,533,DATE
951,688,1,"African American,Latino",138,160,LOCATION
719,504,5,decreasein BTSCC,1151,1166,LOCATION
689,484,8,NCYC,1121,1124,LOCATION
494,337,1,West,334,337,LOCATION


In [0]:
pd_ner_df.ner_label.value_counts()

Out[52]: LOCATION 806
DATE 188
NAME 153
AGE 115
PROFESSION 15
CONTACT 10
ID 9
Name: ner_label, dtype: int64

## BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 1000

ner_df = get_clinical_entities (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+---+-----------+----------------------+-----+---+--------------------+
id |sentence_id|chunk |begin|end|ner_label |
+---+-----------+----------------------+-----+---+--------------------+
0 |0 |human |4 |8 |Organism |
0 |0 |Kir 3.3 |17 |23 |Gene_or_gene_product|
0 |0 |GIRK3 |26 |30 |Gene_or_gene_product|
0 |0 |potassium |92 |100|Simple_chemical |
0 |0 |GIRK |103 |106|Gene_or_gene_product|
0 |1 |chromosome 1q21-23 |188 |205|Cellular_component |
0 |4 |pancreas |697 |704|Organ |
0 |4 |tissues |740 |746|Tissue |
0 |4 |fat andskeletal muscle|749 |770|Tissue |
0 |5 |KCNJ9 |801 |805|Gene_or_gene_product|
0 |5 |Type II |940 |946|Gene_or_gene_product|
1 |0 |breast cancer |84 |96 |Cancer |
1 |0 |patients |134 |141|Organism |
1 |0 |anthracyclines |167 |180|Simple_chemical |
1 |0 |taxanes |186 |192|Simple_chemical |
1 |1 |vinorelbine |246 |256|Simple_chemical |
1 |1 |patients |273 |280|Organism |
1 |1 |breast |309 |314|Cancer |
1 |1 |vinorelbine inpatients|386 |407|Simple_chemical |
1 |1 |anthracyclines |433 |446|Simple_chemical |
+---+-----------+----------------------+-----+---+--------------------+
only showing top 20 rows

## Training a Clinical NER (NCBI Disease Dataset)

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltrain.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/NER_NCBIconlltest.txt
  
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltest.txt", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/NER_NCBIconlltrain.txt", "dbfs:/")

Out[54]: True

In [0]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, 'file:/databricks/driver/NER_NCBIconlltrain.txt')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| pos| label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
Identification of...|[[document, 0, 89...|[[document, 0, 89...|[[token, 0, 13, I...|[[pos, 0, 13, NN,...|[[named_entity, 0...|
The adenomatous p...|[[document, 0, 21...|[[document, 0, 21...|[[token, 0, 2, Th...|[[pos, 0, 2, NN, ...|[[named_entity, 0...|
Complex formation...|[[document, 0, 63...|[[document, 0, 63...|[[token, 0, 6, Co...|[[pos, 0, 6, NN, ...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows

In [0]:
conll_data.count()

Out[56]: 3266

In [0]:
from pyspark.sql import functions as F

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)


+------------+-----+
ground_truth|count|
+------------+-----+
O |75093|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:
conll_data.select("label.result").distinct().count()

Out[58]: 1537

In [0]:
import pyspark.sql.functions as F

#conll_data.select(F.countDistinct("label.result")).show()
#conll_data.groupBy("label.result").count().show(truncate=False)

conll_data = conll_data.withColumn('unique', F.array_distinct("label.result"))\
.withColumn('c', F.size('unique'))\
.filter(F.col('c')>1)

conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)



+------------+-----+
ground_truth|count|
+------------+-----+
O |39427|
I-Disease |3547 |
B-Disease |3093 |
+------------+-----+

In [0]:

# Clinical word embeddings trained on PubMED dataset
clinical_embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]

In [0]:
%sh cd /databricks/driver/ner_graph && ls -l

total 1604
-rw-r--r-- 1 root root 1641016 Oct 30 17:12 blstm_3_200_128_83.pb

In [0]:
test_data = CoNLL().readDataset(spark, "file:/databricks/driver/NER_NCBIconlltest.txt")

test_data = clinical_embeddings.transform(test_data)

test_data.write.parquet('NER_NCBIconlltest.parquet')

In [0]:
!mkdir ner_logs
!mkdir ner_graph
'''!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/nerdl_graph/blstm_3_200_128_83.pb -P /content/ner_graph'''
# !!! Create NERDL Graph with the given functions at the end of this Notebook

dbutils.fs.cp("file:/databricks/driver/ner_graph", "dbfs:/", True)

Out[82]: True

In [0]:

nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(2)\
  .setBatchSize(64)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setTestDataset("NER_NCBIconlltest.parquet")\
  .setOutputLogsPath('ner_logs')\
  .setGraphFolder('file:/databricks/driver/ner_graph')\

ner_pipeline = Pipeline(stages=[
          clinical_embeddings,
          nerTagger
 ])

In [0]:

# 2 epochs 47 sec
ner_model = ner_pipeline.fit(conll_data)

# if you get an error for incompatible TF graph, use 4.1 NerDL-Graph.ipynb notebook to create a graph
# licensed users can also use 17.Graph_builder_for_DL_models.ipynb to create tf graphs easily.

In [0]:
%sh cd /databricks/driver/ner_logs && ls -lt

total 20
-rw-r--r-- 1 root root 1602 Oct 30 17:14 NerDLApproach_4532cbd04d14.log
-rw-r--r-- 1 root root 5047 Oct 30 11:03 NerDLApproach_6f65ff5a5af9.log
-rw-r--r-- 1 root root 3361 Oct 30 09:25 NerDLApproach_e423bc0bed1c.log
-rw-r--r-- 1 root root 1001 Oct 30 09:09 NerDLApproach_4f00276baa1c.log

In [0]:
%sh cat ner_logs/NerDLApproach_4532cbd04d14.log

Name of the selected graph: /databricks/driver/ner_graph/blstm_3_200_128_83.pb
Training started - total epochs: 2 - lr: 0.001 - batch size: 64 - labels: 3 - chars: 81 - training examples: 1348


Epoch 1/2 started, lr: 0.001, dataset size: 1348


Epoch 1/2 - 13.49s - loss: 380.36572 - batches: 23
Quality on validation dataset (20.0%), validation examples = 269
time to finish evaluation: 1.57s
label tp fp fn prec rec f1
I-Disease 204 98 503 0.6754967 0.28854313 0.40436074
B-Disease 44 14 609 0.7586207 0.067381315 0.12376933
tp: 248 fp: 112 fn: 1112 labels: 2
Macro-average prec: 0.71705866, rec: 0.17796223, f1: 0.28515393
Micro-average prec: 0.6888889, rec: 0.18235295, f1: 0.28837207
Quality on test dataset: 
time to finish evaluation: 1.91s
label tp fp fn prec rec f1
I-Disease 180 97 609 0.6498195 0.22813688 0.33771107
B-Disease 36 17 672 0.6792453 0.050847456 0.09461235
tp: 216 fp: 114 fn: 1281 labels: 2
Macro-average prec: 0.6645324, rec: 0.13949217, f1: 0.23058267
Micro-average prec: 0.6545454, rec: 0.14428858, f1: 0.2364532


Epoch 2/2 started, lr: 9.950249E-4, dataset size: 1348


Epoch 2/2 - 8.51s - loss: 194.91808 - batches: 23
Quality on validation dataset (20.0%), validation examples = 269
time to finish evaluation: 1.03s
label tp fp fn prec rec f1
I-Disease 439 197 268 0.6902516 0.62093353 0.65376025
B-Disease 320 117 333 0.7322655 0.49004593 0.587156
tp: 759 fp: 314 fn: 601 labels: 2
Macro-average prec: 0.71125853, rec: 0.5554897, f1: 0.62379694
Micro-average prec: 0.70736253, rec: 0.55808824, f1: 0.6239211

### Evaluate your model

In [0]:
pred_df = ner_model.stages[1].transform(test_data)

In [0]:
pred_df.columns

Out[98]: ['text', 'document', 'sentence', 'token', 'pos', 'label', 'embeddings', 'ner']

In [0]:

from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+-----+-----+-----+---------+------+-----+
 entity| tp| fp| fn|total|precision|recall| f1|
+-------+-----+-----+-----+-----+---------+------+-----+
Disease|372.0|415.0|332.0|704.0| 0.4727|0.5284|0.499|
+-------+-----+-----+-----+-----+---------+------+-----+

+-------------------+
 macro|
+-------------------+
0.49899396378269617|
+-------------------+

None
+------------------+
 micro|
+------------------+
0.4989939637826961|
+------------------+

None

In [0]:
evaler = NerDLMetrics(mode="partial_chunk_per_token", dropO=True)

eval_result = evaler.computeMetricsFromDF(pred_df.select("label","ner"), prediction_col="ner", label_col="label").cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
    .withColumn("recall", F.round(eval_result["recall"],4))\
    .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+-------+-----+----+-----+------+---------+------+----+
 entity| tp| fp| fn| total|precision|recall| f1|
+-------+-----+----+-----+------+---------+------+----+
Disease|798.0|87.0|699.0|1497.0| 0.9017|0.5331|0.67|
+-------+-----+----+-----+------+---------+------+----+

+------------------+
 macro|
+------------------+
0.6700251889168766|
+------------------+

None
+------------------+
 micro|
+------------------+
0.6700251889168766|
+------------------+

None

In [0]:
ner_model.stages[1].write().overwrite().save('/databricks/driver/models/custom_NER_3epoch')

In [0]:

document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

loaded_ner_model = NerDLModel.load("dbfs:/databricks/driver/models/custom_NER_3epoch")\
 .setInputCols(["sentence", "token", "embeddings"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        clinical_embeddings,
        loaded_ner_model,
        converter])

empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [0]:
text = "She has a metastatic breast cancer to lung"

def get_preds(text, light_model):

    result = light_model.fullAnnotate(text)[0]

    return [(i.result, i.metadata['entity']) for i in result['ner_span']]

get_preds(text, light_model)

Out[105]: [('metastatic breast cancer', 'Disease'), ('lung', 'Disease')]

# NERDL Graph

In [0]:
#%tensorflow_version 1.x
 
from sparknlp_jsl.training import tf_graph

tf_graph.print_model_params("ner_dl")

ner_dl parameters.
Parameter Required Default value Description
ntags yes - Number of tags.
embeddings_dim no 200 Embeddings dimension.
nchars no 100 Number of chars.
lstm_size no 128 Number of LSTM units.
gpu_device no 0 Device for training.

In [0]:

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

tf_graph.build("ner_dl", build_params={"embeddings_dim": embeddings_dim, "nchars": nchars, "ntags": ntags}, model_location="./ner_graph", model_filename="auto")

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/sparknlp_jsl/_tf_graph_builders/ner_dl/ner_model.py:12: The name tf.disable_v2_behavior is deprecated. Please use tf.compat.v1.disable_v2_behavior instead.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/python/keras/backend.py:3994: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
WARNING:tensorflow:
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
 * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
 * https://github.com/tensorflow/addons
 * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/contrib/rnn/python/ops/lstm_ops.py:597: Layer.add_variable (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.add_weight` method instead.
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow_core/contrib/crf/python/ops/crf.py:213: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
ner_dl graph exported to ./ner_graph/blstm_3_200_128_83.pb

In [0]:
# for open source users
'''
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/create_graph.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/dataset_encoder.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/ner_model_saver.py
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/training/english/dl-ner/nerdl-graph/sentence_grouper.py

!pip -q install tensorflow==1.15.0

import create_graph

ntags = 3 # number of labels
embeddings_dim = 200
nchars =83

create_graph.create_graph(ntags, embeddings_dim, nchars)
'''

End of Notebook # 1